In [0]:
part_df = spark.read.csv("/FileStore/tables/Cricket_data_set_odi.csv", inferSchema=True, header=True)
part_df.limit(10).display()

Team,Player_ID,Player,Matches,Runs,Wickets,Hundreds,Fifties,Speciality
Ind,498403,Yuvraj Singh,111,2948,167,9,28,All-rounder
SA,533880,Jaq Kallis,248,7758,258,12,38,All-rounder
WI,391674,Dwane Bravo,355,8586,118,5,22,All-rounder
Ban,566945,Shakib Ul-hasan,156,13714,118,11,18,All-rounder
End,10543,Andrew Filntoff,353,18201,216,9,42,All-rounder
Ind,895647,Virat Kohli,343,14970,29,34,66,Bat
Ind,142991,Rohit Sharma,148,8246,10,48,14,Bat
Ind,178781,Shreyas Iyyer,79,10929,21,44,86,Bat
Ind,204027,Shekhar Dhawan,428,15999,4,39,27,Bat
Ind,652081,Virendra Sehwag,426,7758,25,44,65,Bat


In [0]:
%fs
ls /FileStore/tables/csv_write/

path,name,size,modificationTime
dbfs:/FileStore/tables/csv_write/Team/,Team/,0,0
dbfs:/FileStore/tables/csv_write/_SUCCESS,_SUCCESS,0,1700745041000
dbfs:/FileStore/tables/csv_write/_committed_224430566232983173,_committed_224430566232983173,208,1700744717000
dbfs:/FileStore/tables/csv_write/_committed_3444010233655006628,_committed_3444010233655006628,112,1698992595000
dbfs:/FileStore/tables/csv_write/_committed_8144195812023277525,_committed_8144195812023277525,197,1700745041000
dbfs:/FileStore/tables/csv_write/_committed_vacuum2643402836420784351,_committed_vacuum2643402836420784351,96,1700744718000
dbfs:/FileStore/tables/csv_write/_started_224430566232983173,_started_224430566232983173,0,1700744715000
dbfs:/FileStore/tables/csv_write/_started_8144195812023277525,_started_8144195812023277525,0,1700745040000
dbfs:/FileStore/tables/csv_write/part-00000-tid-8144195812023277525-f32e93b4-1d25-424d-b790-db4574233c55-11-1-c000.csv,part-00000-tid-8144195812023277525-f32e93b4-1d25-424d-b790-db4574233c55-11-1-c000.csv,261243,1700745040000


In [0]:
# let's perform partition
# From the above DataFrame, we will be use Team as a partition key
part_df.write.option("header",True)\
    .partitionBy("Team")\
    .mode("overwrite")\
    .csv("/FileStore/tables/csv_write/Team")

In [0]:
%fs
ls /FileStore/tables/csv_write/Team/

path,name,size,modificationTime
dbfs:/FileStore/tables/csv_write/Team/Team=Aus/,Team=Aus/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=Ban/,Team=Ban/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=End/,Team=End/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=Ind/,Team=Ind/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=NZ/,Team=NZ/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=Pak/,Team=Pak/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=SA/,Team=SA/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=SL/,Team=SL/,0,0
dbfs:/FileStore/tables/csv_write/Team/Team=WI/,Team=WI/,0,0
dbfs:/FileStore/tables/csv_write/Team/_SUCCESS,_SUCCESS,0,1701867918000


In [0]:
from pyspark.sql.functions import spark_partition_id

print("Number of partitions-", part_df.rdd.getNumPartitions())
part_df.select(spark_partition_id().alias("part_id")).groupBy("part_id").count().display()
part_df.repartition(4).select(spark_partition_id().alias("part_id")).groupBy("part_id").count().display()

Number of partitions- 1


part_id,count
0,30


part_id,count
0,7
1,8
2,8
3,7


In [0]:
# to perform sql query on partition data 
# we have to create a datafram fro that
partitioned_df = spark.read.csv("/FileStore/tables/csv_write/Team", header=True, inferSchema=True)
partitioned_df.select("*").orderBy("Team").display()
partitioned_df.describe().display()

Player_ID,Player,Matches,Runs,Wickets,Hundreds,Fifties,Speciality,Team
49991,Ricky Pointing,293,7758,36,32,28,Bat,Aus
548172,Mathew Hyden,327,1427,28,37,78,Bat,Aus
519952,Glenn McGrath,254,867,327,0,0,Bow,Aus
16116,Brett Lee,416,1329,266,0,0,Bow,Aus
566945,Shakib Ul-hasan,156,13714,118,11,18,All-rounder,Ban
10543,Andrew Filntoff,353,18201,216,9,42,All-rounder,End
258821,Eoin Morgan,460,7474,9,29,59,Bat,End
174931,Kevin Peterson,332,13368,29,3,58,Bat,End
498403,Yuvraj Singh,111,2948,167,9,28,All-rounder,Ind
895647,Virat Kohli,343,14970,29,34,66,Bat,Ind


summary,Player_ID,Player,Matches,Runs,Wickets,Hundreds,Fifties,Speciality,Team
count,30,30,30,30,30,30,30,30,30
mean,336332.9,null,310.0,8145.3,86.33333333333333,21.633333333333333,39.5,null,null
stddev,242513.10048515047,null,114.14842122852824,5484.047103283713,98.25453690791333,16.826668761498677,30.121306469810964,null,null
min,3956,AB Deviliers,79,867,4,0,0,All-rounder,Aus
max,895647,Yuvraj Singh,461,18201,327,48,88,Bow,WI


In [0]:
# to perform sql queries of dataframe create a temp view
partitioned_df.createOrReplaceTempView("cric_tb")

In [0]:
%sql select * from cric_tb;

Player_ID,Player,Matches,Runs,Wickets,Hundreds,Fifties,Speciality,Team
498403,Yuvraj Singh,111,2948,167,9,28,All-rounder,Ind
895647,Virat Kohli,343,14970,29,34,66,Bat,Ind
142991,Rohit Sharma,148,8246,10,48,14,Bat,Ind
178781,Shreyas Iyyer,79,10929,21,44,86,Bat,Ind
204027,Shekhar Dhawan,428,15999,4,39,27,Bat,Ind
652081,Virendra Sehwag,426,7758,25,44,65,Bat,Ind
153571,Sachin Tendulkar,336,4850,4,40,10,Bat,Ind
540382,MS Dhoni,311,14700,19,26,38,Bat,Ind
550940,Jasprit Bumrah,461,1251,265,0,0,Bow,Ind
97711,Mohmmed Shami,419,5217,210,0,0,Bow,Ind


In [0]:
# start performing your respective quries